<a href="https://colab.research.google.com/github/TristanShoemaker/COMP551-P3/blob/master/InceptionV3_full_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
# from tensorflow.python.keras._impl import keras

from keras.layers import Dense, GlobalAveragePooling2D

from keras.layers import Input
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

import pickle
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

Using TensorFlow backend.


In [14]:
#Loading in the data data
X = pd.read_pickle('train_max_x')
y = pd.read_csv('train_max_y.csv')
y = y['Label']

X = X.astype('float32')/255
X = np.repeat(X.reshape(X.shape[0], 128, 128, 1), 3, axis=3)
y = to_categorical(y, 10)

In [31]:
from keras.applications.vgg16 import VGG16
model = VGG16()
print(model.summary())

553467904/553467096 [==============================] - 352s 1us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________________________

In [33]:
def background_removal(image):
  image[image<1-60/255] = 0
  return image

In [34]:
def show_image(image, id, prediction):
    plt.figure(figsize=(6,5))
    plt.imshow(image, label="", cmap='gray', vmin=0, vmax=1)
    plt.colorbar()

In [35]:
from keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model
#Load VGG along with built-in weights

vgg = VGG16(weights='imagenet',include_top=False, input_shape=(128, 128, 3))

# Freeze the bottom layers
for layer in vgg.layers[:-4]:
    layer.trainable = False

58892288/58889256 [==============================] - 22s 0us/step


In [36]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg)
 
# Add new layers
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# how a summary of the model. Check the number of trainable parameters
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2570      
Total params: 15,898,698
Trainable params: 8,263,434
Non-trainable params: 7,635,264
___________________________________

In [0]:
training_data = datagen.flow(X, y, batch_size=256, subset='training')
print(len(training_data))
validation_data = datagen.flow(X, y, batch_size=batch_size, subset='validation')
history = model.fit_generator(training_data, 
                               epochs=100, verbose=1,
                               validation_data=validation_data)